In [ ]:
%pip install llama-index-readers-file pymupdf
%pip install llama-index-vector-stores-chroma
%pip install llama-index-embeddings-together
%pip install llama-index-llms-llama-cpp

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

api_key=os.getenv("TOEGETHER_API_KEY")

from llama_index.embeddings.together import TogetherEmbedding

embed_model = TogetherEmbedding(
    model_name="togethercomputer/m2-bert-80M-8k-retrieval", api_key=api_key
)

In [3]:
from llama_index.llms.llama_cpp import LlamaCPP

# model_url = "https://huggingface.co/TheBloke/Llama-2-13B-chat-GGML/resolve/main/llama-2-13b-chat.ggmlv3.q4_0.bin"
model_url = "https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q4_0.gguf"

llm = LlamaCPP(
    # You can pass in the URL to a GGML model to download it automatically
    model_url=model_url,
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path=None,
    temperature=0.1,
    max_new_tokens=256,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=3900,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": 1},
    verbose=True,
)

/home/abenittz/workspace/icog-training/rag-app/Rag-training2/venv/lib64/python3.11/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_url" in LlamaCPP has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/home/abenittz/workspace/icog-training/rag-app/Rag-training2/venv/lib64/python3.11/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_path" in LlamaCPP has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/home/abenittz/workspace/icog-training/rag-app/Rag-training2/venv/lib64/python3.11/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_kwargs" in LlamaCPP has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  

total size (MB): 7365.83


  0%|          | 13/7024 [00:54<8:14:19,  4.23s/it]


Download incomplete. Removing partially downloaded file.


ValueError: Download incomplete.

In [4]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
from IPython.display import Markdown, display
import chromadb

In [ ]:
chroma_client = chromadb.EphemeralClient()
chroma_collection = chroma_client.create_collection("quickstart")

In [10]:
from pathlib import Path
from llama_index.readers.file import PyMuPDFReader


loader = PyMuPDFReader()
documents = loader.load(file_path="./data/adhd.pdf")

print(len(documents))

8


In [7]:
from llama_index.core.node_parser import SentenceSplitter

text_parser = SentenceSplitter(
    chunk_size=1024,
    separator=" ",
)

<class 'llama_index.core.node_parser.text.sentence.SentenceSplitter'>


In [11]:

text_chunks = []
# maintain relationship with source doc index, to help inject doc metadata in (3)
doc_idxs = []
for doc_idx, doc in enumerate(documents):
    cur_text_chunks = text_parser.split_text(doc.text)
    text_chunks.extend(cur_text_chunks)
    doc_idxs.extend([doc_idx] * len(cur_text_chunks))
    
print(text_chunks)

['See discussions, stats, and author profiles for this publication at: https://www.researchgate.net/publication/317604534\nThe impact of attention deﬁcit hyperactivity disorder (ADHD) in adulthood: a\nqualitative study\nArticle\xa0\xa0in\xa0\xa0Irish Journal of Psychological Medicine · June 2017\nDOI: 10.1017/ipm.2017.21\nCITATIONS\n18\nREADS\n40,738\n4 authors, including:\nCynthia Watters\n6 PUBLICATIONS\xa0\xa0\xa056 CITATIONS\xa0\xa0\xa0\nSEE PROFILE\nFiona Mcnicholas\nStanford University\n250 PUBLICATIONS\xa0\xa0\xa05,077 CITATIONS\xa0\xa0\xa0\nSEE PROFILE\nBlanaid Gavin\nUniversity College Dublin\n75 PUBLICATIONS\xa0\xa0\xa01,282 CITATIONS\xa0\xa0\xa0\nSEE PROFILE\nAll content following this page was uploaded by Cynthia Watters on 14 September 2017.\nThe user has requested enhancement of the downloaded file.', 'The impact of attention deﬁcit hyperactivity\ndisorder (ADHD) in adulthood: a qualitative study\nC. Watters1, D. Adamis2,*, F. McNicholas3 and B. Gavin3\n1 Psychology Servi